# GMSO - General Molecular Simulation Object

## Using the `Topology`

[GMSO](https://github.com/mosdef-hub/gmso) allows for engine agnostic python object to store everything needed for writing molecule input files for simulation.

In this overview, we will be focusing specifically on the tools we've developed to address the issue of **_system initialization_**, including the creation of a molecular model, the application of a force field (atom-typing and parameter assignment), and output molecular files. The three tools contained within MoSDeF to address system initialization are:


This notebook is designed to provide basics for interfacing with GMSO. Tutorials get more in-depth in terms of complex usage denoted by the `beginners`, `intermediates`, and `experts` tags on these notebooks. For a full list of the examples discussed in these tutorials, please see [the GMSO Tutorials README](readme.md) hub/mosdef_tutorials).

Contained in this notebook is examples of:
* Loading topologies
* Saving topologies


---

In [ ]:
import gmso
import copy
import warnings

warnings.simplefilter("ignore")

## Loading `topologies`
---
Molecular `topologies` are the key component of GMSO, and its generalizeability is emphasized by the number of sources that you can load the `topology` from. 

In [ ]:
# load from files
# mol2, gro, lammps, json, xyz, 
for fn in ["mol2", "gro", "lammps", "json", "xyz"]:
    top = gmso.Topology.load(f"source_files/top.{fn}")
    display(fn+": ", top)


Conversion from python objects

In [ ]:
# mbuild, https://mbuild.mosdef.org/en/stable/topic_guides/load_data.html
from gmso.external import from_mbuild, from_parmed
! wget -O source_files/3UQ6.pdb https://files.rcsb.org/view/3UQ6.pdb
import mbuild as mb
cpd = mb.load("source_files/3UQ6.pdb") # for more info, see other loaders mBuild offers
top = from_mbuild(cpd)
top

In [ ]:
# parmed https://parmed.github.io/ParmEd/html/readwrite.html#writing-files-with-structure-save
from gmso.external import from_parmed
import parmed as pmd
structure = pmd.load_file("source_files/structure.inpcrd", structure=True)
top = from_parmed(structure)
top

In [ ]:
# build internally
from gmso.core.atom import Atom
from gmso.core.bond import Bond
top = gmso.Topology()
c = Atom(name="c")
top.add_site(c)

h1 = Atom(name="h1")
h2 = Atom(name="h2")
h3 = Atom(name="h3")
h4 = Atom(name="h4")
for site in [h1, h2, h3, h4]:
    ch = Bond(connection_members=[c, site])
    top.add_site(site)
    top.add_connection(ch)

top

## Saving `topologies`
---

In [ ]:
# to files
! mkdir saved_files #gromacs, gsd, json, lammps, mcf
top = gmso.Topology.load(f"source_files/top.json")
for fn in ["gro", "lammps", "json", "xyz", "mcf"]:
    outpath = f"saved_files/top.{fn}"
    print(f"Saving out to {outpath}")
    new_top = copy.deepcopy(top) # make sure not to modify topology in place
    new_top.save(outpath, overwrite=True)
    
    

In [ ]:
from gmso.external import to_hoomd_forcefield, to_hoomd_snapshot
import unyt as u
top = gmso.Topology.load(f"source_files/top.json")
hoomd_ff = to_hoomd_forcefield(top, r_cut=1*u.nm)
hoomd_snapshot = to_hoomd_snapshot(top)
display(hoomd_ff, hoomd_snapshot)

In [ ]:
#parmed, mbuild
from gmso.external import to_parmed, to_mbuild
top = gmso.Topology.load(f"source_files/top.json")
structure = to_parmed(top)
cpd = to_mbuild(top)
display(structure, cpd)